In [2]:
#from eps_utils import *
from typing import *

import numpy as np
import pandas as pd

#from  call_bedrock_fast_py import *
from call_bedrock_fast_claude_v3_py import *

In [4]:
pd.read_csv('School_books.csv')

,asin,item_name,gl_product_group,gl_product_group_desc,recommended_browse_node_id1,recommended_browse_node_id2,item_description,product_description,total_ordered_units,Total_Gvs,rbn_eip,bisac_subject_category,thema_classification_code,browse_node_ids_list,sp,branch_validate,leaf_validate
0,B08KGXVK3F,Themes In Indian History Part - 2 For Class - ...,14,gl_book,4149808031,NaN,NaN,NaN,0.0,25.0,4.149808e+09,NaN,NaN,4149808031,['4149808031'],True,False
1,8196168977,Vacation Activity Book Class 1 (A Bridge to Cl...,14,gl_book,4149807031,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,4149807031,['4149807031'],True,False
2,8176139491,CBSE Complete Solutions Sample Papers Geograph...,14,gl_book,4149807031,NaN,NaN,JPH Class 12 Examate Geography Sample Papers C...,0.0,18.0,4.149807e+09,NaN,yp,4149807031,['4149807031'],True,False
3,0733985912,International Mathematics (International Bacca...,14,gl_book,4149838031,4.662559e+09,This is the fifth book in the five book Intern...,NaN,0.0,120.0,4.149838e+09,JUVENILE NONFICTION / Mathematics / General,YPMF,"4149838031,4662559031","['4149838031', '4662559031']",True,False
4,9350951967,Cordova Mastering Grammer for Class 5,14,gl_book,4149808031,4.662554e+09,School textbooks.,NaN,53.0,806.0,4.149808e+09,NaN,NaN,"4149808031,4662554031,4149814031,4662555031","['4149808031', '4662554031', '4149814031', '46...",True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60828,B0CQ585N5M,MA GAYATRI BOOK STORE Junior English Translati...,14,gl_book,4149807031,NaN,NaN,Junior English Translation Book,1.0,17.0,4.149807e+09,NaN,NaN,4149807031,['4149807031'],True,False
60829,9352830881,Self-Learning English Course with Activities -...,14,gl_book,4149808031,4.662554e+09,Self-Learning English Course with Activities i...,NaN,1.0,111.0,4.149808e+09,NaN,NaN,"4149808031,4662554031,4149814031,4662555031","['4149808031', '4662554031', '4149814031', '46...",True,False
60830,2718901713,Liturgie des heures - careme temps pascal - 2,14,gl_book,4149808031,NaN,NaN,NaN,0.0,0.0,4.149808e+09,NaN,NaN,4149808031,['4149808031'],True,False
60831,8130916029,Viva - Albatross Dictionary of Synonyms & Anto...,14,gl_book,4149808031,4.662554e+09,School textbook.,NaN,0.0,4.0,4.149808e+09,NaN,NaN,"4149808031,4662554031","['4149808031', '4662554031']",True,False


In [3]:
entrance_school_prompt_template ='''Human:
<role>
You are an shopping expert who is assessing the categorization of books. Your goal is to determine what is the category of a book given the details of the book.
</role>

<task>

Your task is determine the category of a book which belongs for School Books. The possible categories are mentioned in the <categories></categories> XML tags. You can use the book details given in <book-details></book-details> XML tags to make your decision.
First, use your judgement to think about which category the higher education book belongs to, and output this judgement in <thinking></thinking> XML tags. This is a space for you to write down your thoughts.
Next, output the category you think this book best belongs to in the <category></category> XML tags. This should be copied word-for-word from one of the <item></item> XML tags. If none of the categories match, you should output "NONE OF THE ABOVE" in <category></category> XML tags.
Finally, Mention how confident you are about the predicted category of this book by returning "Very Confident", "Somewhat Confident" or "Not Confident" in <confidence></confidence> XML tags. This should be done after <category></category> XML tags.
Stop responding after </confidence>.
</task>

<categories>
{category}
</categories>


<book-details>
<title>{title}</title>
<description>{description}</description>
</book-details>

Assistant:
<thinking>

'''.strip()

In [4]:
entrance_school_prompt_template1 ='''Human:
<role>
You are an shopping expert who is assessing the categorization of books. Your goal is to determine what is the category of a book given the details of the book.
</role>

<task>

Your task is determine the category of a book which belongs for School Books. The possible categories are mentioned in the <categories></categories> XML tags. You can use the book details given in <product-details></product-details> XML tags to make your decision.
First, use your judgement to think about which category the higher education book belongs to, and output this judgement in <thinking></thinking> XML tags. This is a space for you to write down your thoughts.
Next, output the category you think this book best belongs to in the <category></category> XML tags. This should be copied word-for-word from one of the <item></item> XML tags. If none of the categories match, you should output "NONE OF THE ABOVE" in <category></category> XML tags.
Finally, Mention how confident you are about the predicted category of this book by returning "Very Confident", "Somewhat Confident" or "Not Confident" in <confidence></confidence> XML tags. This should be done after <category></category> XML tags.
Stop responding after </confidence>.
</task>

<categories>
{category}
</categories>

<book-details>
<title>{title}</title>
<description>{description}</description>
<root>{root}</root>
</book-details>



Assistant:
<thinking>

'''.strip()

In [5]:
volt_file=pd.read_csv('44571_mapping(2).txt',sep='\t')
volt_file_hig=volt_file[volt_file['Named path'].str.contains('\[Books, School Books')]
volt_file_hig['Named path'].replace(['\[Books, School Books, ','\]'],"",inplace=True,regex=True)
volt_file_hig=volt_file_hig[['Named path','Classification Key','Is Leaf']]
volt_file_hig.reset_index(drop=True,inplace=True)
volt_file_hig
volt_file_hig['root_node']=volt_file_hig['Named path'].str.split(',',expand=True)[0]

volt_file_hig['root']='<item>'+volt_file_hig['root_node']+'</item>'

root_category="\n".join(list(volt_file_hig['root'].drop_duplicates()))
root_category

/tmp/ipykernel_17781/3325485461.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  volt_file_hig['Named path'].replace(['\[Books, School Books, ','\]'],"",inplace=True,regex=True)
/tmp/ipykernel_17781/3325485461.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volt_file_hig['Named path'].replace(['\[Books, School Books, ','\]'],"",inplace=

'<item>CBSE</item>\n<item>Olympiads & Scholarship Exams</item>\n<item>State Education Boards</item>\n<item>CISCE</item>\n<item>School Atlases</item>\n<item>International School Boards</item>\n<item>Open Schooling</item>\n<item>School Bundles</item>\n<item>[Books</item>'

In [6]:
import pandas as pd
volt_file=pd.read_csv('44571_mapping(2).txt',sep='\t')
volt_file_hig=volt_file[volt_file['Named path'].str.contains('\[Books, School Books')]
volt_file_hig['Named path'].replace(['\[Books, School Books, ','\[','\]'],"",inplace=True,regex=True)
volt_file_hig['root']='<item>'+volt_file_hig['Named path'].str.split(',',expand=True)[0]+'</item>'
volt_file_hig=volt_file_hig[['Named path','Classification Key','root','Is Leaf']]

#volt_file_hig['Leaf Path']=np.where(volt_file_hig['Named path'].str.contains('Engineering Textbooks')==True,volt_file_hig['Named path'],"")
#volt_file_hig_part.replace('Engineering Textbooks, ',"",inplace=True,regex=True)
ass=volt_file_hig['Named path'].str.split(', ',expand=True).drop(columns=0)
volt_file_hig['leaf_1']='<item>'+ass.stack().groupby(level=0).agg(", ".join)+'</item>'
volt_file_hig['Named path2']='<item>'+volt_file_hig['Named path']+'</item>'
volt_file_hig.reset_index(drop=True,inplace=True)
volt_file_hig


volt_file_hig['Named path2']
leaf_category="\n".join(list(volt_file_hig['Named path2'].drop_duplicates()))


/tmp/ipykernel_23977/142448783.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  volt_file_hig['Named path'].replace(['\[Books, School Books, ','\[','\]'],"",inplace=True,regex=True)
/tmp/ipykernel_23977/142448783.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volt_file_hig['Named path'].replace(['\[Books, School Books, ','\[','\]'],"",

In [2]:
from contextlib import contextmanager
from typing import *
@contextmanager
def pd_display(
        max_rows: Optional[int] = None,
        max_cols: Optional[int] = None,
        max_colwidth: Optional[int] = None,
        vertical_align: str = 'top',
        text_align: str = 'left',
        ignore_css: bool = False,
):
    try:
        from IPython.display import display
    except ImportError:
        display = print

    def disp(df: pd.DataFrame):
        css = [
            ## Align header to center
            {
                'selector': 'th',
                'props': [
                    ('vertical-align', 'center'),
                    ('text-align', 'center'),
                    ('padding', '10px'),
                ]
            },
            ## Align cell to top and left
            {
                'selector': 'td',
                'props': [
                    ('vertical-align', vertical_align),
                    ('text-align', text_align),
                    ('padding', '10px'),
                ]
            },

        ]
        if not ignore_css and isinstance(df, pd.DataFrame):
            df = df.style.set_table_styles(css)
        display(df)

    with pd.option_context(
            'display.max_rows', max_rows,
            'display.max_columns', max_cols,
            'max_colwidth', max_colwidth,
            'display.expand_frame_repr', False,
    ):
        yield disp

In [8]:
aud=pd.read_csv('School_books.csv')
aud['description']=np.where(aud['product_description'].isnull()==True,aud['item_description'],aud['product_description'])

In [9]:
def create_root_prompt(row):
    return entrance_school_prompt_template.format(category=root_category,title=row['item_name'],description=row['description'])
    

aud['root_prompt']=aud.apply(create_root_prompt,axis=1)
#aud=aud.head(2000)
aud.reset_index(drop=True,inplace=True)
#aud['prompt']=entrance_exam_prompt_template1.format(
# with pd_display() as disp:
#     disp(aud.sample(3))
display(aud)
print(aud.loc[0,'root_prompt'])

,asin,item_name,gl_product_group,gl_product_group_desc,recommended_browse_node_id1,recommended_browse_node_id2,item_description,product_description,total_ordered_units,Total_Gvs,rbn_eip,bisac_subject_category,thema_classification_code,browse_node_ids_list,sp,branch_validate,leaf_validate,description,root_prompt
0,B08KGXVK3F,Themes In Indian History Part - 2 For Class - ...,14,gl_book,4149808031,NaN,NaN,NaN,0.0,25.0,4.149808e+09,NaN,NaN,4149808031,['4149808031'],True,False,NaN,Human:\n<role>\nYou are an shopping expert who...
1,8196168977,Vacation Activity Book Class 1 (A Bridge to Cl...,14,gl_book,4149807031,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,4149807031,['4149807031'],True,False,NaN,Human:\n<role>\nYou are an shopping expert who...
2,8176139491,CBSE Complete Solutions Sample Papers Geograph...,14,gl_book,4149807031,NaN,NaN,JPH Class 12 Examate Geography Sample Papers C...,0.0,18.0,4.149807e+09,NaN,yp,4149807031,['4149807031'],True,False,JPH Class 12 Examate Geography Sample Papers C...,Human:\n<role>\nYou are an shopping expert who...
3,0733985912,International Mathematics (International Bacca...,14,gl_book,4149838031,4.662559e+09,This is the fifth book in the five book Intern...,NaN,0.0,120.0,4.149838e+09,JUVENILE NONFICTION / Mathematics / General,YPMF,"4149838031,4662559031","['4149838031', '4662559031']",True,False,This is the fifth book in the five book Intern...,Human:\n<role>\nYou are an shopping expert who...
4,9350951967,Cordova Mastering Grammer for Class 5,14,gl_book,4149808031,4.662554e+09,School textbooks.,NaN,53.0,806.0,4.149808e+09,NaN,NaN,"4149808031,4662554031,4149814031,4662555031","['4149808031', '4662554031', '4149814031', '46...",True,False,School textbooks.,Human:\n<role>\nYou are an shopping expert who...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60828,B0CQ585N5M,MA GAYATRI BOOK STORE Junior English Translati...,14,gl_book,4149807031,NaN,NaN,Junior English Translation Book,1.0,17.0,4.149807e+09,NaN,NaN,4149807031,['4149807031'],True,False,Junior English Translation Book,Human:\n<role>\nYou are an shopping expert who...
60829,9352830881,Self-Learning English Course with Activities -...,14,gl_book,4149808031,4.662554e+09,Self-Learning English Course with Activities i...,NaN,1.0,111.0,4.149808e+09,NaN,NaN,"4149808031,4662554031,4149814031,4662555031","['4149808031', '4662554031', '4149814031', '46...",True,False,Self-Learning English Course with Activities i...,Human:\n<role>\nYou are an shopping expert who...
60830,2718901713,Liturgie des heures - careme temps pascal - 2,14,gl_book,4149808031,NaN,NaN,NaN,0.0,0.0,4.149808e+09,NaN,NaN,4149808031,['4149808031'],True,False,NaN,Human:\n<role>\nYou are an shopping expert who...
60831,8130916029,Viva - Albatross Dictionary of Synonyms & Anto...,14,gl_book,4149808031,4.662554e+09,School textbook.,NaN,0.0,4.0,4.149808e+09,NaN,NaN,"4149808031,4662554031","['4149808031', '4662554031']",True,False,School textbook.,Human:\n<role>\nYou are an shopping expert who...


Human:
<role>
You are an shopping expert who is assessing the categorization of books. Your goal is to determine what is the category of a book given the details of the book.
</role>

<task>

Your task is determine the category of a book which belongs for School Books. The possible categories are mentioned in the <categories></categories> XML tags. You can use the book details given in <book-details></book-details> XML tags to make your decision.
First, use your judgement to think about which category the higher education book belongs to, and output this judgement in <thinking></thinking> XML tags. This is a space for you to write down your thoughts.
Next, output the category you think this book best belongs to in the <category></category> XML tags. This should be copied word-for-word from one of the <item></item> XML tags. If none of the categories match, you should output "NONE OF THE ABOVE" in <category></category> XML tags.
Finally, Mention how confident you are about the predicted

In [9]:
aud.reset_index(drop=True,inplace=True)
aud.fillna("",inplace=True)

/tmp/ipykernel_1030/2540733322.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  aud.fillna("",inplace=True)


In [ ]:
bedrock_params = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_new_tokens,
        "messages": [
            {
                "role": "user", 
                "content": prompt,
            }
        ],
    }

In [17]:
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from tqdm import tqdm
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        bedrock_futures = [executor.submit(send_bedrock_request, prompt) for prompt in aud['root_prompt']]
        #boto3_futures = [executor.submit(send_boto3_request, 'i-0123456789abcdef') for _ in range(3)]
        bedrock_results = [future.result() for future in tqdm(bedrock_futures)]
    
#df5['result']=bedrock_results

  0%|          | 86/60833 [01:02<17:29:11,  1.04s/it]

KeyboardInterrupt: 

In [6]:
def call_bedrock(
        *,
        model: str,
        prompt: str,
        max_new_tokens: int,
        temperature: float,
        stop_sequences: Optional[List[str]] = None,
        **kwargs,
) -> Dict:
    start = time.perf_counter()
    bedrock = boto3.client(
        service_name='bedrock-runtime',
        region_name=any_item(['us-east-1', 'us-west-2', 'eu-central-1', 'ap-northeast-1']),
        # endpoint_url='https://bedrock.us-east-1.amazonaws.com',
    )
    bedrock_params = {
        "prompt": prompt, 
        "max_tokens_to_sample": max_new_tokens,
        "temperature": temperature,
    }
    if stop_sequences is not None:
        bedrock_params["stop_sequences"] = stop_sequences
    body = json.dumps(bedrock_params)
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock.invoke_model(body=body, modelId=model, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    end = time.perf_counter()
    return {
        'generated_text': response_body.get('completion'),
        'time_taken_sec': end - start,
    }
bedrock = boto3.client(
  service_name='bedrock-runtime', 
  region_name="us-east-1"
    
)
start=time.time()

modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
def send_bedrock_request(prompt):
    bedrock = boto3.client(
    service_name='bedrock-runtime', 
    region_name=any_item(['us-east-1', 'us-west-2']))
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
      "max_tokens": 500,
        "temperature":0.1,
      "messages": [{"role": "user", "content":prompt }]

    })
    response = bedrock.invoke_model(
        body=body, modelId=modelId
    )
    response_body = json.loads(response.get("body").read())
    
    return response_body.get("content")[0]['text']

def any_item(
        struct: Union[List, Tuple, Set, Dict, str],
        *,
        seed: Optional[int] = None,
        raise_error: bool = True,
) -> Optional[Any]:
    py_random: random.Random = random.Random(seed)
    if isinstance(struct, (set, list, tuple, np.ndarray, pd.Series)) and len(struct) > 0:
        return py_random.choice(tuple(struct))
    elif isinstance(struct, dict):
        k: Any = any_key(struct, seed=seed, raise_error=raise_error)
        v: Any = struct[k]
        return k, v  ## Return an item
    elif isinstance(struct, str):
        return py_random.choice(struct)
    if raise_error:
        raise NotImplementedError(f'Unsupported structure: {type(struct)}')
    return None


In [8]:
def retry(
        fn,
        *args,
        retries: int = 5,
        wait: float = 10.0,
        jitter: float = 0.5,
        silent: bool = True,
        **kwargs
):
    """
    Retries a function call a certain number of times, waiting between calls (with a jitter in the wait period).
    :param fn: the function to call.
    :param retries: max number of times to try. If set to 0, will not retry.
    :param wait: average wait period between retries
    :param jitter: limit of jitter (+-). E.g. jitter=0.1 means we will wait for a random time period in the range
        (0.9 * wait, 1.1 * wait) seconds.
    :param silent: whether to print an error message on each retry.
    :param kwargs: keyword arguments forwarded to the function.
    :return: the function's return value if any call succeeds.
    :raise: RuntimeError if all `retries` calls fail.
    """
    wait: float = float(wait)
    latest_exception = None
    for retry_num in range(retries + 1):
        try:
            return fn(*args, **kwargs)
        except Exception as e:
            latest_exception = traceback.format_exc()
            if not silent:
                print(f'Function call failed with the following exception:\n{latest_exception}')
                if retry_num < (retries - 1):
                    print(f'Retrying {retries - (retry_num + 1)} more times...\n')
            time.sleep(np.random.uniform(wait - wait * jitter, wait + wait * jitter))
    raise RuntimeError(f'Function call failed {retries} times.\nLatest exception:\n{latest_exception}\n')
def call_bedrock_retry(**kwargs):
    return retry(
        send_bedrock_request,
        silent=True,
        retries=10,
        wait=10,
        **kwargs,
    )

In [9]:
#aud['root_prompt']
def call_bedrock_fast(prompts: List[str], **kwargs):
    __executor = ThreadPoolExecutor(max_workers=30)
    try:
        futs = []
        for prompt in prompts:
            futs.append(
                __executor.submit(
                    call_bedrock_retry,
                    prompt=prompt,
                    #modelID=modelId
                    #**kwargs,
                )
            )
        outputs = []
        for prompt, fut in tqdm(zip(prompts, futs), total=len(prompts), ncols=100): 
            outputs.append(fut.result())
        #return pd.Series(outputs)
    finally:
        __executor.shutdown(wait=False)

In [10]:
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
call_bedrock_fast('hello',modelId)

TypeError: call_bedrock_fast() takes 1 positional argument but 2 were given

In [14]:
! pip install bs4
aud['root_output']=outputs
a='\n'.join(list(aud['root_output']))
from bs4 import BeautifulSoup as soup
r=soup(a)
def ext(x,tag_name='p'):
    r=soup(x)
    c=r.find_all(tag_name)
    return c
c=aud['root_output'].apply(ext,tag_name='category')
#print(f'the length of category is:{len(c)}')
#d=aud['root_output'].apply(ext,tag_name='thinking')
#print(f'the length of category is:{len(d)}')
#e=aud['root_output'].apply(ext,tag_name='confidence')
aud['root_Category']=pd.Series(c).astype('str').replace(['<category>','</category>','<item>','</item>','\n','\[','\]'],"",regex=True)
# re['Category'].replace('&amp;','&',inplace=True,regex=True)
# re['Thinking']=pd.Series(d).astype('str').replace(['<thinking>','</thinking>','\n','\[','\]'],"",regex=True)
# aud['root_Category']=r.find_all('category')

aud['root_Category']=aud['root_Category'].astype('str')
aud['root_Category'].replace(['<category>','</category>','<item>','</item>','\n'],"",inplace=True,regex=True)
aud['root_Category'].replace('&amp;','&',inplace=True,regex=True)
#aud=aud.iloc[:,0:33]
aud['root_Category'][0]
aud.to_parquet('root_schoolbooks.parquet')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 7.0 MB/s eta 0:00:00


/tmp/ipykernel_17781/42497198.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  aud['root_Category'].replace(['<category>','</category>','<item>','</item>','\n'],"",inplace=True,regex=True)


In [4]:
aud

NameError: name 'aud' is not defined

In [ ]:
# re=pd.DataFrame()
# root_nodes=aud['root_Category'].drop_duplicates()
# root_nodes=root_nodes.str.strip()
for i in root_nodes:
    df4=aud[aud['root_Category']==i]
   
    df4.reset_index(drop=True,inplace=True)
    leaf_category='\n'.join(list(volt_file_hig[(volt_file_hig['Named path'].str.startswith(i)) & (volt_file_hig['Is Leaf']==True)]['Named path2']))
    def create_root_prompt(row):
        return entrance_school_prompt_template1.format(category=leaf_category,title=row['item_name'],description=row['description'],root=row['root_Category'])
    

    df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
    
    #aud=pd.read_csv('Test set Higher education.csv')
    #aud=aud[['ASIN','item_name','description','generic_item_review','contributor_content']]
    df4.fillna('',inplace=True)
    df4.replace(['<p>','<em>','</p>','</li>','<li>','</em>','<i>','</i>','<DIV>','</DIV>','<div>','</div>','<P>','<STRONG>','</P>','</STRONG>'],"",inplace=True,regex=True)
    model_name='anthropic.claude-v2'
    prompts=df4['leaf_prompt']
    __executor = ThreadPoolExecutor(max_workers=30)
    try:
        futs = []
        for prompt in prompts:
            futs.append(
                __executor.submit(
                    call_bedrock_retry,
                    prompt=prompt,
                    #**kwargs,
                )
            )
        outputs = []
        for prompt, fut in tqdm(zip(prompts, futs), total=len(prompts), ncols=100): 
            outputs.append(fut.result())
        #return pd.Series(outputs)
    finally:
        __executor.shutdown(wait=False)
    df4['leaf_output']=outputs
    
    #     df4['leaf_output']=call_bedrock_fast(prompts=df4['leaf_prompt'], model=model_name,
#                 # model='anthropic.claude-instant-v1',
#                 max_new_tokens=500,
#                 temperature=0.1)
    re=pd.concat([re,df4],axis=0)
    re.astype('str').to_parquet('leaf_output.parquet')
    print(f'Done for {i}')


/tmp/ipykernel_17781/298707191.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
/tmp/ipykernel_17781/298707191.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df4.fillna('',inplace=True)
/tmp/ipykernel_17781/298707191.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.fillna('',inplace=True)
/tmp/ipykernel_17781/29870719

Done for International School Boards


/tmp/ipykernel_17781/298707191.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
/tmp/ipykernel_17781/298707191.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df4.fillna('',inplace=True)
/tmp/ipykernel_17781/298707191.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.fillna('',inplace=True)
/tmp/ipykernel_17781/29870719

Done for State Education Boards


/tmp/ipykernel_17781/298707191.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
/tmp/ipykernel_17781/298707191.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df4.fillna('',inplace=True)
/tmp/ipykernel_17781/298707191.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.fillna('',inplace=True)
/tmp/ipykernel_17781/29870719

...

/tmp/ipykernel_17781/298707191.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
/tmp/ipykernel_17781/298707191.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df4.fillna('',inplace=True)
/tmp/ipykernel_17781/298707191.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.fillna('',inplace=True)
/tmp/ipykernel_17781/29870719

Done for NCERT


/tmp/ipykernel_17781/298707191.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
/tmp/ipykernel_17781/298707191.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df4.fillna('',inplace=True)
/tmp/ipykernel_17781/298707191.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.fillna('',inplace=True)
/tmp/ipykernel_17781/29870719

Done for Books


100%|█████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.44s/it]

In [3]:
re=pd.read_parquet('leaf_output.parquet')


In [7]:
# from bs4 import BeautifulSoup as soup
# #r=soup(a)
# re['leaf_output1']=re['leaf_output'].apply(lambda x:"<thinking>"+str(x))
# def ext(x,tag_name='p'):
#     r=soup(x)
#     c=r.find_all(tag_name)
#     return c
# c=re['leaf_output1'].apply(ext,tag_name='category')
# #print(f'the length of category is:{len(c)}')
# d=re['leaf_output1'].apply(ext,tag_name='thinking')
# #print(f'the length of category is:{len(d)}')
# e=re['leaf_output1'].apply(ext,tag_name='confidence')
# re['Category']=pd.Series(c).astype('str').replace(['<category>','</category>','<item>','</item>','\n','\[','\]'],"",regex=True)
# re['Category'].replace('&amp;','&',inplace=True,regex=True)
# re['Thinking']=pd.Series(d).astype('str').replace(['<thinking>','</thinking>','\n','\[','\]'],"",regex=True)
# re['Confidence']=pd.Series(e).astype('str').replace(['<confidence>','</confidence>','\n','\[','\]'],"",regex=True)
re1=pd.merge(re,volt_file_hig[['Named path','Classification Key']],left_on='Category',right_on='Named path',how='left')
print(len(re1))
print("-"*5)
#df2['leaf_validation']=df2['Classification Key']==df2['Recommended_Nodes']
re1.drop(columns='Named path',inplace=True)
re1

60795
-----


,asin,item_name,gl_product_group,gl_product_group_desc,recommended_browse_node_id1,recommended_browse_node_id2,item_description,product_description,total_ordered_units,Total_Gvs,...,root_prompt,root_output,root_Category,leaf_prompt,leaf_output,leaf_output1,Category,Thinking,Confidence,Classification Key
0,B08KGXVK3F,Themes In Indian History Part - 2 For Class - ...,14,gl_book,4149808031,,,,0.0,25.0,...,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this appea...",CBSE,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this appea...","<thinking>Based on the book details provided, ...","CBSE, Textbooks","Based on the book details provided, this appea...",Very Confident,2.303410e+10
1,8196168977,Vacation Activity Book Class 1 (A Bridge to Cl...,14,gl_book,4149807031,,,,0.0,0.0,...,Human:\n<role>\nYou are an shopping expert who...,"<thinking>\nBased on the book title ""Vacation ...",CBSE,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this appea...","<thinking>Based on the book details provided, ...","CBSE, Reference Books","Based on the book details provided, this appea...",Somewhat Confident,2.303409e+10
2,8176139491,CBSE Complete Solutions Sample Papers Geograph...,14,gl_book,4149807031,,,JPH Class 12 Examate Geography Sample Papers C...,0.0,18.0,...,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this book ...",CBSE,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this book ...","<thinking>Based on the book details provided, ...","CBSE, Question Papers","Based on the book details provided, this book ...",Very Confident,2.303409e+10
3,0199482217,Springboard UKG Semester 1,14,gl_book,4149808031,4662554031.0,Springboard—An Integrated Semester Course for ...,,1.0,65.0,...,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this book ...",CBSE,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this book ...","<thinking>Based on the book details provided, ...","CBSE, Textbooks","Based on the book details provided, this book ...",Very Confident,2.303410e+10
4,9380644388,Vishesh Hindi Vyakaran Tatha Rachna 4,14,gl_book,4149808031,,Vishesh Hindi Vyakaran Tatha Rachna 4,,12.0,125.0,...,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this book ...",CBSE,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this book ...","<thinking>Based on the book details provided, ...","CBSE, Textbooks","Based on the book details provided, this book ...",Very Confident,2.303410e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60790,8121905184,Advanced Practical Zoology,14,gl_book,4662474031,4149808031.0,Reading books is a kind of enjoyment. Reading ...,,14.0,325.0,...,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this book ...",Books,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this appea...","<thinking>Based on the book details provided, ...",Higher Education Textbooks,"Based on the book details provided, this appea...",Very Confident,NaN
60791,8183502709,HAPPINESS A TREASURE WITHIN PART 1,14,gl_book,4149807031,,,School books,1.0,37.0,...,Human:\n<role>\nYou are an shopping expert who...,"Based on the book details provided, this book ...",Books,Human:\n<role>\nYou are an shopping expert who...,"Based on the description ""School books"" for th...","<thinking>Based on the description ""School boo...",NONE OF THE ABOVE,"Based on the description ""School books"" for th...",Very Confident,NaN
60792,9352753852,Scholastic Classics: The Happy Prince,14,gl_book,4149814031,1318159031.0,"""The living always think that gold can make th...",,3.0,56.0,...,Human:\n<role>\nYou are an shopping expert who...,"Based 

In [10]:
re1[['asin','item_name','description','Thinking','Category','Classification Key']].to_csv('School_books.csv',index=False)

In [21]:
# model_name='anthropic.claude-3-sonnet-20240229-v1:0'
# aud['root_output']=call_bedrock_fast(prompts=aud['root_prompt'],
#                 # model='anthropic.claude-instant-v1',
#                )

In [ ]:
bedrock_params = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_new_tokens,
        "messages": [
            {
                "role": "user", 
                "content": prompt,
            }
        ],
    }

In [9]:
import boto3
import json

bedrock = boto3.client(service_name="bedrock-runtime",)
body = json.dumps({
  "max_tokens": 256,
  "messages": [{"role": "user", "content": "Hello, world"}],
  "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0")

response_body = json.loads(response.get("body").read())
print(response_body.get("content"))

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeModel operation: User: arn:aws:sts::174305766568:assumed-role/CatxSpectrumRole/SageMaker is not authorized to perform: bedrock:InvokeModel on resource: arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0 because no identity-based policy allows the bedrock:InvokeModel action

In [19]:
import boto3
prom='Consider yourself as a Ml Scinentist. You have been given a task to create solution around time series. There is a Data frame where there are 10 skus impression data for 12 months. You need to\
take the same as an input and create a time series forecasting solution using RNN to predict the impressions for next 12 months for all 10 listings. Create a sample data set for your reference and \
provide the end to end solution.Please output the data in <out></out> xml tags'

bedrock = boto3.client(
  service_name='bedrock-runtime', 
  region_name=any_item(['us-east-1', 'us-west-2'])
    
)
start=time.time()
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

def send_bedrock_request(prompt):
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
      "max_tokens": 2000,
        "temperature":0.1,
      "messages": [{"role": "user", "content":prompt }]

    })
    response = bedrock.invoke_model(
        body=body, modelId=modelId
    )
    response_body = json.loads(response.get("body").read())
    
    return response_body.get("content")[0]['text']
send_bedrock_request(prom)

"<out>\n# Import necessary libraries\nimport pandas as pd\nimport numpy as np\nfrom sklearn.preprocessing import MinMaxScaler\nfrom keras.models import Sequential\nfrom keras.layers import Dense, LSTM, Dropout\n\n# Create a sample dataset\ndata = {\n    'SKU': ['SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU6', 'SKU7', 'SKU8', 'SKU9', 'SKU10'],\n    'Month1': [1000, 2500, 3200, 1800, 2000, 1500, 2800, 3100, 1900, 2700],\n    'Month2': [1200, 2700, 3000, 2100, 1900, 1600, 2900, 3300, 2000, 2600],\n    'Month3': [1100, 2600, 3400, 1900, 2100, 1700, 3000, 3200, 1800, 2800],\n    'Month4': [1300, 2800, 3300, 2200, 2000, 1800, 3100, 3400, 2100, 2700],\n    'Month5': [1400, 2900, 3500, 2000, 2200, 1900, 3200, 3300, 1900, 2900],\n    'Month6': [1200, 2700, 3300, 2300, 2100, 1700, 3000, 3500, 2200, 2800],\n    'Month7': [1500, 3000, 3600, 2100, 2300, 2000, 3300, 3400, 2000, 3000],\n    'Month8': [1300, 2800, 3400, 2400, 2200, 1800, 3100, 3600, 2300, 2900],\n    'Month9': [1600, 3100, 3700, 2200, 

In [20]:
print('''\n# Import necessary libraries\nimport pandas as pd\nimport numpy as np\nfrom sklearn.preprocessing import MinMaxScaler\nfrom keras.models import Sequential\nfrom keras.layers import Dense, LSTM, Dropout\n\n# Create a sample dataset\ndata = {\n    'SKU': ['SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU6', 'SKU7', 'SKU8', 'SKU9', 'SKU10'],\n    'Month1': [1000, 2500, 3200, 1800, 2000, 1500, 2800, 3100, 1900, 2700],\n    'Month2': [1200, 2700, 3000, 2100, 1900, 1600, 2900, 3300, 2000, 2600],\n    'Month3': [1100, 2600, 3400, 1900, 2100, 1700, 3000, 3200, 1800, 2800],\n    'Month4': [1300, 2800, 3300, 2200, 2000, 1800, 3100, 3400, 2100, 2700],\n    'Month5': [1400, 2900, 3500, 2000, 2200, 1900, 3200, 3300, 1900, 2900],\n    'Month6': [1200, 2700, 3300, 2300, 2100, 1700, 3000, 3500, 2200, 2800],\n    'Month7': [1500, 3000, 3600, 2100, 2300, 2000, 3300, 3400, 2000, 3000],\n    'Month8': [1300, 2800, 3400, 2400, 2200, 1800, 3100, 3600, 2300, 2900],\n    'Month9': [1600, 3100, 3700, 2200, 2400, 2100, 3400, 3500, 2100, 3100],\n    'Month10': [1400, 2900, 3500, 2500, 2300, 1900, 3200, 3700, 2400, 3000],\n    'Month11': [1700, 3200, 3800, 2300, 2500, 2200, 3500, 3600, 2200, 3200],\n    'Month12': [1500, 3000, 3600, 2600, 2400, 2000, 3300, 3800, 2500, 3100]\n}\n\ndf = pd.DataFrame(data)\n\n# Separate features and target\nfeatures = df.drop(['SKU'], axis=1)\ntarget = features.copy()\n\n# Normalize the data\nscaler = MinMaxScaler()\nfeatures_scaled = scaler.fit_transform(features)\n\n# Split data into train and test sets\ntrain_size = int(len(features_scaled) * 0.8)\ntrain_data = features_scaled[:train_size, :]\ntest_data = features_scaled[train_size:, :]\n\n# Create input and output sequences\nlook_back = 1\nX_train, y_train = create_sequences(train_data, look_back)\nX_test, y_test = create_sequences(test_data, look_back)\n\n# Define RNN model\nmodel = Sequential()\nmodel.add(LSTM(64, input_shape=(look_back, features.shape[1])))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(features.shape[1]))\nmodel.compile(optimizer='adam', loss='mse')\n\n# Train the model\nmodel.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))\n\n# Make predictions for the next 12 months\nfuture_periods = 12\nfuture_data = test_data[-look_back:]\nfuture_predictions = []\n\nfor _ in range(future_periods):\n    x_input = future_data.reshape(1, look_back, features.shape[1])\n    y_pred = model.predict(x_input)\n    future_predictions.append(y_pred[0])\n    future_data = np.append(future_data[1:], y_pred, axis=0)\n\n# Inverse transform the predictions\nfuture_predictions_unscaled = scaler.inverse_transform(np.array(future_predictions).reshape(-1, features.shape[1]))\n\n# Create a DataFrame for the predictions\npredictions_df = pd.DataFrame(future_predictions_unscaled, columns=features.columns)\npredictions_df.insert(0, 'SKU', df['SKU'])\n\nprint(predictions_df)\n</out>\n\nThis code provides an end-to-end solution for time series forecasting using an RNN (Recurrent Neural Network) model. Here's a breakdown of the steps:\n\n1. Import necessary libraries.\n2. Create a sample dataset with 10 SKUs and 12 months of impression data.\n3. Separate the features (impression data) and target (same as features for time series forecasting).\n4. Normalize the data using MinMaxScaler.\n5. Split the data into train and test sets.\n6. Create input and output sequences for the RNN model.\n7. Define the RNN model architecture using Keras (LSTM layer, Dropout, and Dense output layer).\n8. Train the model on the training data.\n9. Make predictions for the next 12 months using the trained model.\n10. Inverse transform the predictions to get the actual impression values.\n11. Create a DataFrame with the predictions for the next 12 months, including the SKU column.\n12. Print the predictions DataFrame.\n\nNote: The `create_sequences` function is not included in the code snippet, but it should be implemented to create input and output sequences from the data for the RNN model.''')


# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Create a sample dataset
data = {
    'SKU': ['SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU6', 'SKU7', 'SKU8', 'SKU9', 'SKU10'],
    'Month1': [1000, 2500, 3200, 1800, 2000, 1500, 2800, 3100, 1900, 2700],
    'Month2': [1200, 2700, 3000, 2100, 1900, 1600, 2900, 3300, 2000, 2600],
    'Month3': [1100, 2600, 3400, 1900, 2100, 1700, 3000, 3200, 1800, 2800],
    'Month4': [1300, 2800, 3300, 2200, 2000, 1800, 3100, 3400, 2100, 2700],
    'Month5': [1400, 2900, 3500, 2000, 2200, 1900, 3200, 3300, 1900, 2900],
    'Month6': [1200, 2700, 3300, 2300, 2100, 1700, 3000, 3500, 2200, 2800],
    'Month7': [1500, 3000, 3600, 2100, 2300, 2000, 3300, 3400, 2000, 3000],
    'Month8': [1300, 2800, 3400, 2400, 2200, 1800, 3100, 3600, 2300, 2900],
    'Month9': [1600, 3100, 3700, 2200, 2400, 2100, 3400, 3500, 2

In [3]:
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
ab=call_bedrock_fast(['hello','hi'],model=modelId,temperature=0.1,max_new_tokens=100)
ab

100%|█████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.63it/s]


0    Hello! How can I assist you today?
1                                Hello!
dtype: object